In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from zipfile import ZipFile

!pip3 install datetime
from datetime import datetime as dt

In [2]:
# https://www.programiz.com/python-programming/datetime/strftime

date_created = dt.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
date_created

'2021-06-04-18-26-35-991057'

In [3]:
spark = SparkSession \
    .builder \
    .appName("spark_local") \
    .getOrCreate()

# Load Songs Data

In [4]:
with ZipFile('data/song-data.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall('data/song-data')

In [5]:
song_data_df = spark.read.json("data/song-data/*/*/*/*/*.json")

In [6]:
song_data_df.printSchema()
song_data_df.limit(5).toPandas()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARDR4AC1187FB371A1,NaN,,NaN,Montserrat Caballé;Placido Domingo;Vicente Sar...,511.16363,1,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,0
1,AREBBGV1187FB523D2,NaN,"Houston, TX",NaN,Mike Jones (Featuring CJ_ Mello & Lil' Bran),173.66159,1,SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),0
2,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
3,ARPBNLO1187FB3D52F,40.71455,"New York, NY",-74.00712,Tiny Tim,43.36281,1,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,2000
4,ARDNS031187B9924F0,32.67828,Georgia,-83.22295,Tim Wilson,186.48771,1,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,2005


In [7]:
song_pandas = song_data_df.toPandas()
song_pandas['song_id'].nunique()

71

# Create Song and Artist Tables
## Output tables in Parquet Files

In [8]:
# Create Song Data Table parquet file
song_data_df.createOrReplaceTempView("song_data_DF")

song_table = spark.sql("""
    SELECT  DISTINCT song_id, 
            title, 
            artist_id, 
            year, 
            duration
    FROM song_data_DF
    ORDER BY song_id
    """)

song_table.printSchema()
song_table.limit(5).toPandas()

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)



,song_id,title,artist_id,year,duration
0,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,ARPBNLO1187FB3D52F,2000,43.36281
1,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,ARDR4AC1187FB371A1,0,511.16363
2,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,ARMAC4T1187FB3FA4C,2004,207.77751
3,SOBBXLX12A58A79DDA,Erica (2005 Digital Remaster),AREDBBQ1187B98AFF5,0,138.63138
4,SOBCOSW12A8C13D398,Rumba De Barcelona,AR7SMBG1187B9B9066,0,218.38322


In [9]:
song_table.describe().show()

+-------+------------------+--------------------+------------------+-----------------+------------------+
|summary|           song_id|               title|         artist_id|             year|          duration|
+-------+------------------+--------------------+------------------+-----------------+------------------+
|  count|                71|                  71|                71|               71|                71|
|   mean|              null|                null|              null|785.9577464788732|239.72967605633815|
| stddev|              null|                null|              null|980.9571191533842|106.56277912134072|
|    min|SOAOIBZ12AB01815BE|A Higher Place (A...|AR051KA1187B98B2FF|                0|          29.54404|
|    max|SOZVMJI12AB01808AF|   ¿Dónde va Chichi?|ARYKCQI1187FB3B18F|             2008|         599.24853|
+-------+------------------+--------------------+------------------+-----------------+------------------+



In [10]:
import os
if not os.path.exists('data/output_data'):
    os.makedirs('data/output_data')

In [11]:
#https://sparkbyexamples.com/pyspark/pyspark-read-and-write-parquet-file/
    

song_table.write.mode("overwrite").partitionBy("year", "artist_id").parquet("/data/output_data/song_table.parquet")

In [12]:
song_table_read = spark.read.parquet("/data/output_data/song_table.parquet")

In [13]:
song_table_read.printSchema()
song_table_read.limit(5).toPandas()

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)



,song_id,title,duration,year,artist_id
0,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,43.36281,2000,ARPBNLO1187FB3D52F
1,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,186.48771,2005,ARDNS031187B9924F0
2,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),326.00771,0,ARLTWXK1187FB5A3F8
3,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),267.70240,0,ARKFYS91187B98E58F
4,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),279.97995,0,ARPFHN61187FB575F6


In [14]:
song_table_read.describe().show()

+-------+------------------+--------------------+------------------+-----------------+------------------+
|summary|           song_id|               title|          duration|             year|         artist_id|
+-------+------------------+--------------------+------------------+-----------------+------------------+
|  count|                71|                  71|                71|               71|                71|
|   mean|              null|                null|239.72967605633804|785.9577464788732|              null|
| stddev|              null|                null|106.56277912134071|980.9571191533839|              null|
|    min|SOAOIBZ12AB01815BE|A Higher Place (A...|          29.54404|                0|AR051KA1187B98B2FF|
|    max|SOZVMJI12AB01808AF|   ¿Dónde va Chichi?|         599.24853|             2008|ARYKCQI1187FB3B18F|
+-------+------------------+--------------------+------------------+-----------------+------------------+



In [15]:
song_data_df.createOrReplaceTempView("artist_data_DF")

artist_table = spark.sql("""
    SELECT
        DISTINCT artist_id,
                 artist_name as name,
                 artist_location as location,
                 artist_latitude as latitude,
                 artist_longitude as longitude
        FROM artist_data_DF
        ORDER BY artist_id
                 """)

artist_table.printSchema()
artist_table.limit(5).toPandas()

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



,artist_id,name,location,latitude,longitude
0,AR051KA1187B98B2FF,Wilks,,NaN,NaN
1,AR0IAWL1187B9A96D0,Danilo Perez,Panama,8.41770,-80.11278
2,AR0RCMP1187FB3F427,Billie Jo Spears,"Beaumont, TX",30.08615,-94.10158
3,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
4,AR1Y2PT1187FB5B9CE,John Wesley,Brandon,27.94017,-82.32547


In [16]:
artist_table.write.mode("overwrite").parquet("/data/output_data/artist_table.parquet")

In [17]:
artist_table_df = spark.read.parquet("/data/output_data/artist_table.parquet")

artist_table_df.printSchema()
artist_table_df.limit(5).toPandas()

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



,artist_id,name,location,latitude,longitude
0,ARDR4AC1187FB371A1,Montserrat Caballé;Placido Domingo;Vicente Sar...,,NaN,NaN
1,ARMAC4T1187FB3FA4C,The Dillinger Escape Plan,"Morris Plains, NJ",40.82624,-74.47995
2,ARNF6401187FB57032,Sophie B. Hawkins,"New York, NY [Manhattan]",40.79086,-73.96644
3,AROUOZZ1187B9ABE51,Willie Bobo,"New York, NY [Spanish Harlem]",40.79195,-73.94512
4,ARI2JSK1187FB496EF,Nick Ingman;Gavyn Wright,"London, England",51.50632,-0.12714


In [18]:
# Load Log Data

In [19]:
# https://thispointer.com/python-how-to-unzip-a-file-extract-single-multiple-or-all-files-from-a-zip-archive/
# Unzip the log_data file into new folder
with ZipFile('data/log-data.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall('data/log-data')


In [20]:
log_data_df = spark.read.json("data/log-data/*.json")

In [21]:
log_data_df.printSchema()
log_data_df.limit(5).toPandas()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12


In [22]:
log_data_filtered_df = log_data_df.filter(log_data_df.page == 'NextSong')

log_data_filtered_df.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


In [23]:
log_data_filtered_df.createOrReplaceTempView("user_table_DF")

user_table = spark.sql("""
    SELECT DISTINCT int(userId) as user_id,
                    firstName as first_name,
                    lastName as last_name,
                    gender,
                    level
    FROM user_table_DF
    ORDER BY user_id
                    """)
user_table.printSchema()
user_table.limit(5).toPandas()

root
 |-- user_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)



,user_id,first_name,last_name,gender,level
0,2,Jizelle,Benjamin,F,free
1,3,Isaac,Valdez,M,free
2,4,Alivia,Terrell,F,free
3,5,Elijah,Davis,M,free
4,6,Cecilia,Owens,F,free


In [24]:
user_table.write.mode("overwrite").partitionBy("user_id").parquet("/data/output_data/user_table.parquet")

In [25]:
user_table = spark.read.parquet('/data/output_data/user_table.parquet')

user_table.printSchema()
user_table.limit(5).toPandas()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)
 |-- user_id: integer (nullable = true)



,first_name,last_name,gender,level,user_id
0,Mohammad,Rodriguez,M,free,88
1,Mohammad,Rodriguez,M,paid,88
2,Jacqueline,Lynch,F,free,29
3,Jacqueline,Lynch,F,paid,29
4,Matthew,Jones,M,paid,36


In [26]:
#https://stackoverflow.com/questions/51983037/convert-from-timestamp-to-specific-date-in-pyspark

from pyspark.sql.functions import udf
from datetime import datetime
from pyspark.sql import types as t
# Create a function that returns the desired string from a timestamp

def format_timestamp(ts):
    return datetime.fromtimestamp(ts / 1000.0)

# Create the UDF
format_timestamp_udf = udf(lambda x: format_timestamp(x), t.TimestampType())

# Finally, apply the function to each element of the 'timestamp' column
log_data_filtered_df = log_data_filtered_df.withColumn('timestamp', format_timestamp_udf(log_data_filtered_df['ts']))

In [27]:
log_data_filtered_df.printSchema()
log_data_filtered_df.limit(5).toPandas()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:30:26.796
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:41:21.796
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:45:41.796
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,2018-11-15 03:44:09.796
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:48:55.796


In [28]:
#https://stackoverflow.com/questions/51983037/convert-from-timestamp-to-specific-date-in-pyspark

from pyspark.sql.functions import udf
from datetime import datetime
from pyspark.sql import types as t
# Create a function that returns the desired string from a timestamp

def format_datetime(ts):
    return datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

# Create the UDF
format_datetime_udf = udf(lambda x: format_datetime(x), t.StringType())

# Finally, apply the function to each element of the 'timestamp' column
log_data_filtered_df = log_data_filtered_df.withColumn('datetime', format_datetime_udf(log_data_filtered_df['ts']))

In [29]:
log_data_filtered_df.printSchema()
log_data_filtered_df.limit(5).toPandas()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- datetime: string (nullable = true)



,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp,datetime
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:30:26.796,2018-11-15 00:30:26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:41:21.796,2018-11-15 00:41:21
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:45:41.796,2018-11-15 00:45:41
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,2018-11-15 03:44:09.796,2018-11-15 03:44:09
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:48:55.796,2018-11-15 05:48:55


In [30]:
log_data_filtered_df.createOrReplaceTempView("time_table_DF")

time_table = spark.sql("""
    SELECT DISTINCT datetime as start_time,
            hour(timestamp) as hour,
            day(timestamp) as day,
            weekofyear(timestamp) as week,
            month(timestamp) as month,
            year(timestamp) as year,
            dayofweek(timestamp) as weekday
    FROM time_table_DF
    ORDER BY start_time """)

time_table.printSchema()
time_table.limit(5).toPandas()

root
 |-- start_time: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)



,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46,21,1,44,11,2018,5
1,2018-11-01 21:05:52,21,1,44,11,2018,5
2,2018-11-01 21:08:16,21,1,44,11,2018,5
3,2018-11-01 21:11:13,21,1,44,11,2018,5
4,2018-11-01 21:17:33,21,1,44,11,2018,5


In [31]:
time_table.write.mode("overwrite").partitionBy("year", "month").parquet("/data/output_data/time_table.parquet")

In [32]:
time_table = spark.read.parquet("/data/output_data/time_table.parquet")

time_table.printSchema()
time_table.limit(5).toPandas()

root
 |-- start_time: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



,start_time,hour,day,week,weekday,year,month
0,2018-11-05 18:34:59,18,5,45,2,2018,11
1,2018-11-05 18:36:22,18,5,45,2,2018,11
2,2018-11-05 18:37:12,18,5,45,2,2018,11
3,2018-11-05 18:41:01,18,5,45,2,2018,11
4,2018-11-05 18:41:34,18,5,45,2,2018,11


In [33]:
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/

song_log_join_df = song_data_df.join(log_data_filtered_df, (log_data_filtered_df.artist == song_data_df.artist_name) & (log_data_filtered_df.song == song_data_df.title))

song_log_join_df.printSchema()
song_log_join_df.limit(5).toPandas()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year,...,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp,datetime
0,AR5KOSW1187FB35FF4,49.80388,Dubai UAE,15.47491,Elena,269.58322,1,SOZCTXZ12AB0182364,Setanta matins,0,...,NextSong,1.541048e+12,818,Setanta matins,200,1542837407796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15,2018-11-21 21:56:47.796,2018-11-21 21:56:47


In [34]:
# https://stackoverflow.com/questions/46213986/how-could-i-add-a-column-to-a-dataframe-in-pyspark-with-incremental-values

from pyspark.sql.functions import monotonically_increasing_id

song_log_join_df = song_log_join_df.withColumn("songplay_id", monotonically_increasing_id())

song_log_join_df.createOrReplaceTempView('songplay_table_df')

songplay_table = spark.sql("""
    SELECT  songplay_id,
            datetime as start_time,
            userId as user_id,
            level,
            song_id,
            artist_id,
            sessionId as session_id,
            location,
            userAgent as user_agent
    FROM songplay_table_df
    ORDER BY songplay_id
            """)

songplay_table.printSchema()
songplay_table.limit(5).toPandas()

root
 |-- songplay_id: long (nullable = false)
 |-- start_time: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)



,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,0,2018-11-21 21:56:47,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."


In [ ]:
songplay_table.write.mode("overwrite").partitionBy("year", "month").parquet("/data/output_data/songplay_table.parquet")

In [ ]:
songplay_table = spark.read.parquet("/data/output_data/songplay_table.parquet")

songplay_table.printSchema()
songplay_table.limit(5).toPandas()